In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
df = pd.read_csv('/home/nicoli/github/ssd_keras/dataset/csv/imdb_csv/imdb.csv', index_col=0)

In [3]:
df_sorted = df.sort_values(by='celeb_id', axis=0)

In [4]:
df_sorted.isnull().sum()

celeb_id                  0
name                      0
full_path                 0
face_score                0
second_face_score    242679
xmin                      0
xmax                      0
ymin                      0
ymax                      0
dob_date                  0
photo_taken               0
age                       0
gender                    0
dtype: int64

## Remover itens com rostos não identificados

In [5]:
df_sorted.drop(df_sorted.loc[df_sorted['face_score']==-np.inf].index, inplace=True)

In [6]:
df_sorted.shape, df.shape

((391087, 13), (452132, 13))

## Remover itens com mais de um rosto identificado

In [7]:
df_sorted_final = df_sorted.drop(df_sorted.loc[df_sorted['second_face_score'].notnull()].index)

In [8]:
df_sorted_final.shape

(181634, 13)

## Pegar descrição da quantidade de itens por celebridaade

In [10]:
df_sorted_final.groupby('celeb_id').size().describe()

count    14624.000000
mean        12.420268
std         25.980946
min          1.000000
25%          1.000000
50%          3.000000
75%         10.000000
max        395.000000
dtype: float64

## Remover itens de celebridades com mais de 50 imagens

In [20]:
keys = df_sorted_final.groupby('celeb_id').groups.keys()

In [63]:
def drop_celeb_ids_items(df_celeb_id, qtd):
    if df_celeb_id.shape[0] >50:
        return df_celeb_id.sample(df_celeb_id.shape[0] - 50).index

In [64]:
celeb_ids = df_sorted_final.groupby('celeb_id').size()[df_sorted_final.groupby('celeb_id').size()>50].index

In [65]:
df_sorted_50 = df_sorted_final.copy()

In [66]:
for celeb_id in celeb_ids:
    df_celeb_id = df_sorted_final.groupby('celeb_id').get_group(celeb_id)
    df_sorted_50.drop(drop_celeb_ids_items(df_celeb_id,50), axis=0, inplace=True)

In [69]:
df_sorted_50.groupby('celeb_id').size()[df_sorted_50.groupby('celeb_id').size()>50]

Series([], dtype: int64)

## Remover idades negativas

In [75]:
df_sorted_50.drop(df_sorted_50[df_sorted_50.age<0].index, inplace=True)

## Salvar dfs

In [73]:
df_sorted_final.to_csv('/home/nicoli/github/ssd_keras/dataset/csv/imdb_csv/imdb_clean.csv')

In [72]:
df_sorted_50.to_csv('/home/nicoli/github/ssd_keras/dataset/csv/imdb_csv/imdb_clean_50.csv')

In [73]:
df_sorted_50.shape, df_sorted_final.shape

((140964, 13), (181634, 13))